In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [2]:
# Load data
data_path = "../../data"
X_train = pd.read_csv(os.path.join(data_path, "X_train.csv")).values
y_train_org = pd.read_csv(os.path.join(data_path, "y_train.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test.csv")).values
y_test_org = pd.read_csv(os.path.join(data_path, "y_test.csv"))
traits = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness']

In [3]:
# Create results directory
results_path = "../../results"
specific_results_path = os.path.join("../../results", "cnn_classification")
os.makedirs(results_path, exist_ok=True)
os.makedirs(specific_results_path, exist_ok=True)

In [4]:
def calc_roc_auc(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n_classes = np.unique(y_true)
    roc_auc_scores = []
    for label in n_classes:
        # Create binary labels for the current class vs. all other classes
        y_true_class = (y_true == label).astype(int)
        y_pred_class = (y_pred == label).astype(int)
        
        # Calculate ROC AUC for the current class
        roc_auc = roc_auc_score(y_true_class, y_pred_class)
        roc_auc_scores.append(roc_auc)
    return roc_auc_scores

In [5]:
for trait in traits:
    print(f"Processing {trait}")
    trait_bin = trait + "_bin"
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
    y_train = np.array([label_mapping[label] for label in y_train_org[trait_bin]])
    y_test = np.array([label_mapping[label] for label in y_test_org[trait_bin]])
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

    num_classes = len(np.unique(y_test)) 

    checkpoint = ModelCheckpoint(f"best_model_{trait}.h5", 
                                monitor='val_acc',  
                                verbose=1, 
                                save_best_only=True,    
                                mode='max',             
                                save_weights_only=False) 
    # Define the CNN model
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.5),  # Add dropout to reduce overfitting
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(3, activation='softmax')  # Adjust the output layer units based on your problem
    ])

    # Create a Sequential model
    # model = keras.Sequential()
    # model.add(layers.Input(shape=(X_train.shape[1],)))
    # # Add hidden layers with different activation functions
    # model.add(layers.Dense(256, activation='relu'))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(128, activation='sigmoid'))
    # model.add(layers.Dense(64, activation='sigmoid'))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dense(16, activation='relu'))
    # # Output layer with 3 classes and softmax activation for classification
    # model.add(layers.Dense(3, activation='softmax'))

    # # Output layer with 3 units (3 classes) and softmax activation
    # model.add(layers.Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',  # Use this loss for multiclass classification
                metrics=['accuracy'])

    # Display the model summary
    model.summary()

    # Train the model
    batch_size = 32
    epochs = 100

    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=0.2,
                        callbacks=[checkpoint])  # Pass the checkpoint callback

    # Save the trained model to a file
    model.save(f"best_model_{trait}.h5")

Processing Extraversion
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2560      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11011 (43.01 KB)
Trainable params: 11011 (43.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
47/47 [==============================] - 0s 4ms/step - loss: 1.0356 - accuracy: 0.4829 - val_loss: 0.9937 - val_accu

/Users/jauyeung/Library/Caches/pypoetry/virtualenvs/ascertain-investigation-iYp4vF4l-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


47/47 [==============================] - 0s 3ms/step - loss: 0.7636 - accuracy: 0.5632 - val_loss: 0.7277 - val_accuracy: 0.4893
Epoch 2/100
47/47 [==============================] - 0s 1ms/step - loss: 0.7035 - accuracy: 0.5605 - val_loss: 0.7058 - val_accuracy: 0.4893
Epoch 3/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6842 - accuracy: 0.5773 - val_loss: 0.7008 - val_accuracy: 0.5000
Epoch 4/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6848 - accuracy: 0.5759 - val_loss: 0.7072 - val_accuracy: 0.5053
Epoch 5/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6831 - accuracy: 0.5753 - val_loss: 0.6806 - val_accuracy: 0.5802
Epoch 6/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6818 - accuracy: 0.5866 - val_loss: 0.6935 - val_accuracy: 0.5241
Epoch 7/100
47/47 [==============================] - 0s 1ms/step - loss: 0.6816 - accuracy: 0.5860 - val_loss: 0.6811 - val_accuracy: 0.5615
Epoch 8/100
47/47 [======

In [6]:
for trait in traits:
    best_model = tf.keras.models.load_model(f"best_model_{trait}.h5")
    test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy}")

7/7 [==============================] - 0s 1ms/step - loss: 0.9180 - accuracy: 0.5817
Test Accuracy: 0.5817307829856873
7/7 [==============================] - 0s 1ms/step - loss: 0.8183 - accuracy: 0.5240
Test Accuracy: 0.5240384340286255
7/7 [==============================] - 0s 1ms/step - loss: 0.9510 - accuracy: 0.3558
Test Accuracy: 0.35576921701431274
7/7 [==============================] - 0s 1ms/step - loss: 1.0478 - accuracy: 0.6154
Test Accuracy: 0.6153846383094788
7/7 [==============================] - 0s 894us/step - loss: 0.6025 - accuracy: 0.6923
Test Accuracy: 0.692307710647583
